Back to **[Fan](https://fanwangecon.github.io/)**'s R4Econ Homepage **[Table of Content](https://fanwangecon.github.io/R4Econ/)**

# Generate Joint Quantiles from Multiple Continuous Variables as a Categorical Variable with Linear Index

There are multiple or a single continuous variables. Find which quantile each observation belongs to for each of the variables. Then also generate a joint/interaction variable of all combinations of quantiles from different variables.

The program has these features:

1. Quantiles breaks are generated based on group_by characteristics, meaning quantiles for individual level characteristics when data is panel
2. Quantiles variables apply to full panel at within-group observation levels.
3. Robust to non-unique breaks for quantiles (non-unique grouped together)
4. Quantile categories have detailed labeling (specifying which non-unique groupings belong to quantile)

## Program

## Use Program

### Load Data

In [13]:
# Library
library(tidyverse)

# Load Sample Data
setwd('C:/Users/fan/R4Econ/_data/')
df <- read_csv('height_weight.csv')

Parsed with column specification:
cols(
  S.country = col_character(),
  vil.id = col_double(),
  indi.id = col_double(),
  sex = col_character(),
  svymthRound = col_double(),
  momEdu = col_double(),
  wealthIdx = col_double(),
  hgt = col_double(),
  wgt = col_double(),
  hgt0 = col_double(),
  wgt0 = col_double(),
  prot = col_double(),
  cal = col_double(),
  p.A.prot = col_double(),
  p.A.nProt = col_double()
)


## Line by Line

The idea of the function is to generate quantiles levels first, and then use those to generate the categories based on quantiles. Rather than doing this in one step. These are done in two steps, to increase clarity in the quantiles used for quantile category generation. And a dataframe with these quantiles are saved as a separate output of the function.

### Dataframe of Variables' Group-by Level Quantiles

Quantiles from Different Variables. Note that these variables are specific to the individual, not individual/month. So we need to first slick the data, so that we only get the first rows. 

Do this in several steps to clarify group_by level. No speed loss. 

In [14]:
# These are the continuous variables over which to generate quantiles
vars.group_by <- c('indi.id')
vars.arrange <- c('indi.id', 'svymthRound')
vars.continuous <- c('wealthIdx', 'hgt0', 'wgt0')
seq.quantiles <- seq(0,1,0.10)
seq.quantiles <- c(0, 0.25, 0.50, 0.75, 1)
seq.quantiles <- c(0, 0.3333, 0.6666, 1.0)
# seq.quantiles <- c(0, 0.50, 1)
# Quantiles for any variable
gen_quantiles <- function(var, df, prob=c(0.25, 0.50, 0.75)) {
    enframe(quantile(as.numeric(df[[var]]), prob, na.rm=TRUE), 'quant.perc', var) 
}

In [15]:
# Selected Variables, many Percentiles
df_indiL1 <- df %>% group_by(!!!syms(vars.group_by)) %>% arrange(!!!syms(vars.arrange)) %>% slice(1L)
dim(df_indiL1)
df.quant.vars.cts <- lapply(vars.continuous, gen_quantiles, df=df_indiL1, prob=seq.quantiles) %>% reduce(full_join)
t(df.quant.vars.cts)

[1] 2023   15

Joining, by = "quant.perc"
Joining, by = "quant.perc"


quant.perc,0%,33.33%,66.66%,100%
wealthIdx,0.5,0.5,3.0,14.0
hgt0,40.9,48.6,50.3,59.0
wgt0,1410.200,2885.528,3221.898,5180.900


In [16]:
# Quantiles all Variables
df.quant.all <- suppressMessages(lapply(names(df), gen_quantiles, df=df_indiL1, prob=seq(0.1,0.9,0.10)) %>% reduce(full_join))
df.quant.all

Warning message in quantile(as.numeric(df[[var]]), prob, na.rm = TRUE):
"NAs introduced by coercion"Warning message in quantile(as.numeric(df[[var]]), prob, na.rm = TRUE):
"NAs introduced by coercion"

quant.perc,S.country,vil.id,indi.id,sex,svymthRound,momEdu,wealthIdx,hgt,wgt,hgt0,wgt0,prot,cal,p.A.prot,p.A.nProt
10%,NA,3,203.2,NA,0,0.50,0.5,46.3,1463.92,46.6,2508.83,0.5,0.50,29.30,0.50
20%,NA,4,405.4,NA,0,0.50,0.5,47.6,1959.00,47.7,2700.78,0.5,0.50,201.22,0.50
30%,NA,6,607.6,NA,0,0.50,0.5,48.3,2336.59,48.4,2830.69,0.5,0.50,727.55,5.39
40%,NA,8,809.8,NA,0,4.10,0.9,48.9,2751.72,48.9,2954.12,0.5,0.50,1021.64,45.46
50%,NA,9,1012.0,NA,0,7.60,1.9,49.4,3146.40,49.4,3052.00,0.5,0.50,1261.30,121.85
60%,NA,12,1214.2,NA,0,11.92,2.8,50.0,3496.64,50.0,3157.76,0.5,4.12,1577.08,228.06
70%,NA,14,1416.4,NA,0,16.00,3.1,50.7,3846.83,50.5,3282.13,0.7,8.30,2395.17,260.70
80%,NA,17,1618.6,NA,0,20.00,4.1,51.4,4313.78,51.1,3446.40,1.1,12.10,4285.28,293.06
90%,NA,26,1820.8,NA,0,23.78,6.0,52.4,4719.33,52.0,3698.51,1.5,16.20,9019.84,368.14


### Cut Quantile Categorical Variables

Using the Quantiles we have generate, cut the continuous variables to generate categorical quantile variables in the full dataframe.

Note that we can only cut based on unique breaks, but sometimes quantile break-points are the same if some values are often observed, and also if there are too few observations with respect to quantile groups. 

To resolve this issue, we only look at unique quantiles. 

We need several support Functions:
1. support functions to generate suffix for quantile variables based on quantile cuts
2. support for labeling variables of resulting quantiles beyond bracketing

In [17]:
# Support Functions for Variable Suffix 
f_Q_suffix <- function(seq.quantiles) {
    quantile.suffix <- paste0('Qs', min(seq.quantiles),
                              'e', max(seq.quantiles),
                              'n', (length(seq.quantiles)-1))
}
# Support Functions for Quantile Labeling
f_Q_label <- function(arr.quantiles, 
                      arr.sort.unique.quantile,
                      seq.quantiles) {
    paste0('(', 
           paste0(which(arr.quantiles %in% arr.sort.unique.quantile), collapse=','), 
           ') of ', f_Q_suffix(seq.quantiles)) 
}
# Function Testing
arr.quantiles <- df.quant.vars.cts[[substitute('wealthIdx')]]
arr.quantiles
arr.sort.unique.quantiles <- sort(unique(df.quant.vars.cts[[substitute('wealthIdx')]]))
arr.sort.unique.quantiles
f_Q_label(arr.quantiles, arr.sort.unique.quantiles[1], seq.quantiles)
f_Q_label(arr.quantiles, arr.sort.unique.quantiles[2], seq.quantiles)
lapply(arr.sort.unique.quantiles[1:(length(arr.sort.unique.quantiles)-1)],
       f_Q_label,
       arr.quantiles=arr.quantiles,
       seq.quantiles=seq.quantiles)

[1]  0.5  0.5  3.0 14.0

[1]  0.5  3.0 14.0

[1] "(1,2) of Qs0e1n3"

[1] "(3) of Qs0e1n3"

[[1]]
[1] "(1,2) of Qs0e1n3"

[[2]]
[1] "(3) of Qs0e1n3"

In [18]:
# Cutting Function, Cut Continuous Variables into Quantiles with labeing
f_cut <- function(var, df.quant.vars, include.lowest=TRUE, fan.labels=TRUE) {
    
    # unparsed string variable name
    var.str <- substitute(var)
    
    # Breaks
    arr.quantiles <- df.quant.vars[[var.str]]
    arr.sort.unique.quantiles <- sort(unique(arr.quantiles))
    print(arr.sort.unique.quantiles)
    
    # Regular cutting With Standard Labels
    # TRUE, means the lowest group has closed bracket left and right 
    var.quantile <- cut(var, breaks=arr.sort.unique.quantiles, include.lowest=include.lowest)
    
    # Use my custom labels
    if (fan.labels) {
        levels.suffix <- lapply(arr.sort.unique.quantiles[1:(length(arr.sort.unique.quantiles)-1)],
                                f_Q_label,
                                arr.quantiles=arr.quantiles,
                                seq.quantiles=seq.quantiles)
        print(levels.suffix)
        levels(var.quantile) <- paste0(levels(var.quantile), '; ', levels.suffix)
    }
    
    # Return
    return(var.quantile)
}
# Generate New Variable Names with Quantile Suffix
f_var_rename <- function(name, seq.quantiles) {
    quantile.suffix <- paste0('_', f_Q_suffix(seq.quantiles))
    return(sub('_q', quantile.suffix, name))
}

In [19]:
# Generate Categorical Variables of Quantiles
df.new <- df %>% mutate_at(vars.continuous,
                           funs(q=f_cut(., df.quant.vars.cts, 
                                       include.lowest=TRUE, fan.labels=TRUE))) %>% 
            rename_at(vars(contains('_q')), 
                      funs(f_var_rename(., seq.quantiles=seq.quantiles)))

[1]  0.5  3.0 14.0
[[1]]
[1] "(1,2) of Qs0e1n3"

[[2]]
[1] "(3) of Qs0e1n3"

[1] 40.9 48.6 50.3 59.0
[[1]]
[1] "(1) of Qs0e1n3"

[[2]]
[1] "(2) of Qs0e1n3"

[[3]]
[1] "(3) of Qs0e1n3"

[1] 1410.200 2885.528 3221.898 5180.900
[[1]]
[1] "(1) of Qs0e1n3"

[[2]]
[1] "(2) of Qs0e1n3"

[[3]]
[1] "(3) of Qs0e1n3"



### Individual Variables' Quantile Cuts Review Results

In [20]:
# Group By Results
f.count <- function(df, var.cts) {
    df %>% select(S.country, indi.id, svymthRound, matches(paste0(vars.continuous, collapse='|'))) %>%
        group_by(!!sym(f_var_rename(paste0(var.cts,'_q'), seq.quantiles))) %>%
        summarise_all(funs(n=n()))
}

In [21]:
# Full Panel Results
lapply(vars.continuous, f.count, df=df.new)

wealthIdx_Qs0e1n3,S.country_n,indi.id_n,svymthRound_n,wealthIdx_n,hgt0_n,wgt0_n,hgt0_Qs0e1n3_n,wgt0_Qs0e1n3_n
"[0.5,3]; (1,2) of Qs0e1n3",23258,23258,23258,23258,23258,23258,23258,23258
"(3,14]; (3) of Qs0e1n3",11784,11784,11784,11784,11784,11784,11784,11784
NA,7,7,7,7,7,7,7,7
hgt0_Qs0e1n3,S.country_n,indi.id_n,svymthRound_n,wealthIdx_n,hgt0_n,wgt0_n,wealthIdx_Qs0e1n3_n,wgt0_Qs0e1n3_n
"[40.9,48.6]; (1) of Qs0e1n3",10476,10476,10476,10476,10476,10476,10476,10476
"(48.6,50.3]; (2) of Qs0e1n3",9724,9724,9724,9724,9724,9724,9724,9724
"(50.3,59]; (3) of Qs0e1n3",9597,9597,9597,9597,9597,9597,9597,9597
NA,5252,5252,5252,5252,5252,5252,5252,5252
wgt0_Qs0e1n3,S.country_n,indi.id_n,svymthRound_n,wealthIdx_n,hgt0_n,wgt0_n,wealthIdx_Qs0e1n3_n,hgt0_Qs0e1n3_n
"[1.41e+03,2.89e+03]; (1) of Qs0e1n3",10032,10032,10032,10032,10032,10032,10032,10032


In [22]:
# Results Individual Slice
lapply(vars.continuous, f.count, 
       df=(df.new %>% group_by(!!!syms(vars.group_by)) %>% arrange(!!!syms(vars.arrange)) %>% slice(1L)))

wealthIdx_Qs0e1n3,S.country_n,indi.id_n,svymthRound_n,wealthIdx_n,hgt0_n,wgt0_n,hgt0_Qs0e1n3_n,wgt0_Qs0e1n3_n
"[0.5,3]; (1,2) of Qs0e1n3",1366,1366,1366,1366,1366,1366,1366,1366
"(3,14]; (3) of Qs0e1n3",657,657,657,657,657,657,657,657
hgt0_Qs0e1n3,S.country_n,indi.id_n,svymthRound_n,wealthIdx_n,hgt0_n,wgt0_n,wealthIdx_Qs0e1n3_n,wgt0_Qs0e1n3_n
"[40.9,48.6]; (1) of Qs0e1n3",594,594,594,594,594,594,594,594
"(48.6,50.3]; (2) of Qs0e1n3",552,552,552,552,552,552,552,552
"(50.3,59]; (3) of Qs0e1n3",549,549,549,549,549,549,549,549
NA,328,328,328,328,328,328,328,328
wgt0_Qs0e1n3,S.country_n,indi.id_n,svymthRound_n,wealthIdx_n,hgt0_n,wgt0_n,wealthIdx_Qs0e1n3_n,hgt0_Qs0e1n3_n
"[1.41e+03,2.89e+03]; (1) of Qs0e1n3",565,565,565,565,565,565,565,565
"(2.89e+03,3.22e+03]; (2) of Qs0e1n3",564,564,564,564,564,564,564,564
